# \Sigma_{a1} в зависимости от плотности теплоносителя, температуры теплоносителя, температуры топлива и концентрации борной кислоты

## 1. Загрузка данных

In [ ]:
import pandas as pd

Загрузка данных

In [ ]:
df = pd.read_excel('Input/Data.xlsx')
df.head(3)

In [ ]:
names = ['\\rho(g/cm^3)',
 'T_c(K)',
 'T_f(K)',
 'c_b(ppm)',
 '\\Sigma_{a1}(3)']

In [ ]:
train_dataset = df[names].sample(frac=0.8,random_state=0)
test_dataset = df[names].drop(train_dataset.index)

In [ ]:
train_dataset.head(3)

In [ ]:
test_dataset.head(3)

## 2. Визуализация данных

### 2.1 Корреляционная матрица

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(figsize=(5,5)) # Sample figsize in inches
sns.heatmap(train_dataset[names].corr(), annot = True, fmt='.2g')

### 2.2 Диаграммы рассеяния

In [ ]:
import seaborn as sns
sns.set_theme(color_codes=True)

ax = sns.regplot(x=df['\\rho(g/cm^3)'],
                 y="\Sigma_{a1}(3)",
                 data=df,
                 scatter_kws={'s':7, "color": "black", 'alpha':0.5},
                 line_kws={"color": "red"}, order = 2)

In [ ]:
import seaborn as sns
sns.set_theme(color_codes=True)

ax = sns.regplot(x=df['T_c(K)'],
                 y="\Sigma_{a1}(3)",
                 data=df,
                 scatter_kws={'s':7, "color": "black", 'alpha':0.5},
                 line_kws={"color": "red"}, order = 1)

In [ ]:
import seaborn as sns
sns.set_theme(color_codes=True)

ax = sns.regplot(x=df['T_f(K)'],
                 y="\Sigma_{a1}(3)",
                 data=df,
                 scatter_kws={'s':7, "color": "black", 'alpha':0.5},
                 line_kws={"color": "red"}, order = 1)

In [ ]:
import seaborn as sns
sns.set_theme(color_codes=True)

ax = sns.regplot(x=df['c_b(ppm)'],
                 y="\Sigma_{a1}(3)",
                 data=df,
                 scatter_kws={'s':7, "color": "black", 'alpha':0.5},
                 line_kws={"color": "red"}, order = 1)

## 3. Линейная регрессия

In [ ]:
import statsmodels.api as sm
import statsmodels as statsmodels

train_data = train_dataset[names[:-1]]
train_data = sm.add_constant(train_data)

test_data = test_dataset[names[:-1]]
test_data = sm.add_constant(test_data)

display(train_data.head(3))
display(test_data.head(3))

In [ ]:
train_labels = train_dataset['\\Sigma_{a1}(3)']
display(train_labels.head(3))

test_labels = test_dataset['\\Sigma_{a1}(3)']
display(test_labels.head(3))

In [ ]:
model = sm.OLS(train_labels, train_data)

fit = model.fit()
g = fit.summary()
g

In [ ]:
g = fit.pvalues
g

T_c(K) - не значим на 10-ом % урвоне значимости

In [ ]:
features = ['\\rho(g/cm^3)', 'T_f(K)', 'c_b(ppm)']

In [ ]:
train_data = train_dataset[features]
train_data = sm.add_constant(train_data)

train_data.head(3)

In [ ]:
model = sm.OLS(train_labels, train_data)

fit = model.fit()
g = fit.summary()
g

In [ ]:
p = fit.pvalues
p

Все регрессоры значимы на 1-ом уровне значимости %.

In [ ]:
ypred = fit.predict(train_data)
ypred

In [ ]:
train_labels

Mean squared error

\begin{align*}
\text{MSE} = \frac{1}{N} \text{RSS} = \frac{1}{N} \sum (f_i -y_i)^2
\end{align*}

In [ ]:
MSE = statsmodels.tools.eval_measures.mse(train_labels, ypred)
MSE

Root mean squared error

\begin{align*}
\sigma = \sqrt{MSE}
\end{align*}

In [ ]:
RMSE = statsmodels.tools.eval_measures.rmse(train_labels, ypred)
RMSE

Root Mean Squared Percentage Error

In [ ]:
RMSPE = statsmodels.tools.eval_measures.rmspe(train_labels, ypred)
RMSPE

Root Mean Squared Percentage Error for test data

In [ ]:
test_data = test_dataset[features]
test_data = sm.add_constant(test_data)
ypred = fit.predict(test_data)

RMSPE = statsmodels.tools.eval_measures.rmspe(test_labels, ypred)
RMSPE

## 4. Deep learning

### 4.1 Нормализация признаков

In [ ]:
features = ['\\rho(g/cm^3)',
 'T_c(K)',
 'T_f(K)',
 'c_b(ppm)']

In [ ]:
df[features].describe()

In [ ]:
df_features_norm = df.copy()

for i in range(len(features)):
    df_features_norm[features[i]] = (df[features[i]]-df[features[i]].mean())/df[features[i]].std()

In [ ]:
df_features_norm[features].describe()

### 4.2 Нейросетевые модели

### 4.2.1 Подготовка к моделированию

In [ ]:
names

In [ ]:
df_features_norm = df_features_norm[names]
df_features_norm.head(3)

In [ ]:
df_features_norm.describe()

In [ ]:
features

In [ ]:
df_features_norm[features].head(3)

Разделим данные на обучающую и тестовую выборки

In [ ]:
train_dataset = df_features_norm.sample(frac=0.8,random_state=0)
test_dataset = df_features_norm.drop(train_dataset.index)

In [ ]:
train_dataset.head(3)

In [ ]:
test_dataset.head(3)

Отделим признаки от меток

In [ ]:
train_labels = train_dataset.pop('\Sigma_{a1}(3)')
test_labels = test_dataset.pop('\Sigma_{a1}(3)')

In [ ]:
train_dataset.head(2)

In [ ]:
train_labels.head(2)

Модель

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

def build_model(learning_rate, rho, activation):
    model = keras.Sequential([
        layers.Dense(4, activation=activation, input_shape=[len(train_dataset.keys())]),
        layers.Dense(2, activation=activation),
        layers.Dense(1)
    ])
    
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate, rho = rho)
    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])
    return model

In [ ]:
# Выведем прогресс обучения в виде точек после каждой завершенной эпохи
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:
            print('')
        print('.', end='')

In [ ]:
learning_rate = 0.001
rho = 0.99
activation = 'tanh'
model = build_model(learning_rate, rho, activation)

model.summary()

In [ ]:
normed_train_data = train_dataset

In [ ]:
normed_train_data.head(3)

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

Обучение модели

In [ ]:
EPOCHS = 100

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

Визуализируем процесс обучения модели используя статистику содержащуюся в объекте history

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_history(history, ymax):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error')
    plt.plot(hist['epoch'],
             hist['mse'],
             label='Train Error')
    plt.plot(hist['epoch'],
             hist['val_mse'],
             label = 'Val Error')
    plt.ylim([0,ymax])
    plt.legend()
    plt.show()

plot_history(history, 0.003)

In [ ]:
train_labels.head(10)

In [ ]:
model.predict(normed_train_data)[0:10]

### 4.2.2 Модель 1

In [ ]:
# построение модели

learning_rate = 0.001
rho = 0.9
activation = 'tanh'
model = build_model(learning_rate, rho, activation)

model.summary()

In [ ]:
# Обучение модели

EPOCHS = 300

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
# История обучения

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
# Отображение процесса обучения

plot_history(history, 0.0003)

In [ ]:
# Тренировочные данные

train_labels.head(10)

In [ ]:
# Предсказанные данные

model.predict(normed_train_data)[0:10]

### 4.2.3 Модель 2

In [ ]:
def build_model_1_layer(learning_rate, rho, activation):
    model = keras.Sequential([
        layers.Dense(4, activation=activation, input_shape=[len(train_dataset.keys())]),
        layers.Dense(1)
    ])
    
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate, rho = rho)
    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])
    return model

In [ ]:
learning_rate = 0.0002
rho = 0.95
activation = 'tanh'
model = build_model_1_layer(learning_rate, rho, activation)

model.summary()

In [ ]:
# Обучение модели

EPOCHS = 1200

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
# История обучения

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
# Отображение процесса обучения

plot_history(history, 0.00003)

In [ ]:
# Тренировочные данные

train_labels.head(10)

In [ ]:
# Предсказанные данные

model.predict(normed_train_data)[0:10]

### 4.2.4 Модель 3

In [ ]:
def build_model_0_layer(learning_rate, rho, activation):
    model = keras.Sequential([
        layers.Dense(1, activation=activation, input_shape=[len(train_dataset.keys())])
    ])
    
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate, rho = rho)
    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])
    return model

In [ ]:
learning_rate = 0.0002
rho = 0.95
activation = 'tanh'
model = build_model_0_layer(learning_rate, rho, activation)

model.summary()

In [ ]:
# Обучение модели

EPOCHS = 1200

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
# История обучения

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail(10)

In [ ]:
# Тренировочные данные

train_labels.head(10)

In [ ]:
# Предсказанные данные

model.predict(normed_train_data)[0:10]

### 4.2.5 Модель 4

In [ ]:
normed_train_data

In [ ]:
normed_train_data = normed_train_data.drop(['T_c(K)'], axis=1)
normed_train_data

In [ ]:
def build_model_0_layer(learning_rate, rho, activation):
    model = keras.Sequential([
        layers.Dense(1, activation=activation, input_shape=[len(normed_train_data.keys())])
    ])
    
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate, rho = rho)
    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])
    return model

In [ ]:
learning_rate = 0.0002
rho = 0.9999
activation = 'tanh'
model = build_model_0_layer(learning_rate, rho, activation)

model.summary()

In [ ]:
# Обучение модели

EPOCHS = 1200

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
# История обучения

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail(10)

In [ ]:
# Отображение процесса обучения

plot_history(history, 0.00000001)

In [ ]:
# Тренировочные данные

train_labels.head(10)

In [ ]:
# Предсказанные данные

model.predict(normed_train_data)[0:10]